# HRRR Explorer

In [ ]:
import xarray as xr
import panel as pn
import metpy
import hvplot.xarray
from cartopy import crs as ccrs
from geoviews import tile_sources as gvts

In [ ]:
url = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/HRRR/CONUS_2p5km/Best'

In [ ]:
update_button = pn.widgets.Button(name='Load latest data', button_type='success')

In [ ]:
def plot(ds, crs, var, base_map):
    extra_dims = list(ds[var].dims[:-2])
    da = ds[var].unify_chunks().load()
    mesh = da.hvplot.quadmesh(x='x', y='y', rasterize=True, crs=crs, title=var,
                                   attr_labels=False, project=True,
                                   groupby=extra_dims, cmap='turbo',
                                   frame_height=400, frame_width=600).opts(alpha=0.7,
                                   data_aspect=None, active_tools=['wheel_zoom', 'pan'])
    return pn.panel(mesh * base_map, widgets={k: pn.widgets.Select for k in extra_dims})

In [ ]:
def map_select(event=None):
    ds = xr.open_dataset(url, chunks={'time':1})

    time_vars = []
    for var in ds.data_vars:
        if len(ds[var].dims) > 0:
            if 'time' in ds[var].dims[0] and not 'bounds' in var:
                time_vars.append(var)

    init_var = 'Temperature_height_above_ground'
    ds = ds.metpy.parse_cf()
    crs = ds[init_var].metpy.cartopy_crs

    var_select = pn.widgets.Select(name='HRRR Variables:', options=sorted(time_vars), value=init_var)
    base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)
    
    plot_app = pn.bind(plot, ds=ds, crs=crs, var=var_select, base_map=base_map_select)
    return pn.Column(
        var_select,
        base_map_select,
        plot_app
    )

In [ ]:
app = pn.Column(
    update_button,
    pn.bind(map_select, update_button)
)

In [ ]:
app.servable('HRRR Explorer')